In [9]:
from typing import Optional

from langchain_community.graphs import Neo4jGraph

import os
from dotenv import load_dotenv


load_dotenv()


NEO4J_URI = "bolt://localhost:7687"  # Update if your Neo4j URI is different
NEO4J_USER = os.getenv("NEO4J_USER")  # Update with your Neo4j username
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")  # Update with your Neo4j password

graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USER,
    password=NEO4J_PASSWORD,
    refresh_schema=False
)

In [2]:
import glob
from langchain_core.documents import Document

# Get all .txt files in the ../data/scrapped/ folder
file_paths = glob.glob("../data/scrapped/*.txt")

# Read each file and create a Document
documents = []
for file_path in file_paths:
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
        documents.append(Document(page_content=text))

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini')

In [5]:
from langchain_experimental.graph_transformers import LLMGraphTransformer

node_properties = True
relationship_properties = True

no_schema = LLMGraphTransformer(
    llm=llm,
    node_properties=node_properties,
    relationship_properties=relationship_properties
)

In [6]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

async def main():
    data = await no_schema.aconvert_to_graph_documents(documents)
    return data

# Ensure the coroutine is awaited properly
data = asyncio.get_event_loop().run_until_complete(main())

ClientError: {code: Neo.ClientError.Procedure.ProcedureNotFound} {message: There is no procedure with the name `apoc.merge.node` registered for this database instance. Please ensure you've spelled the procedure name correctly and that the procedure is properly deployed.}

In [7]:
data

[GraphDocument(nodes=[Node(id='Spending', type='Concept', properties={}), Node(id='Saving', type='Concept', properties={}), Node(id='Financial Emergencies', type='Goal', properties={}), Node(id='College', type='Goal', properties={}), Node(id='Retirement', type='Goal', properties={}), Node(id='New Car', type='Goal', properties={}), Node(id='Home Down Payment', type='Goal', properties={}), Node(id='Vacation', type='Goal', properties={}), Node(id='Business Launch', type='Goal', properties={}), Node(id='Outstanding Debts', type='Concept', properties={}), Node(id='Credit Card Debt', type='Concept', properties={}), Node(id='Emergency Fund', type='Concept', properties={}), Node(id='Living Expenses', type='Concept', properties={}), Node(id='Liquid Account', type='Concept', properties={}), Node(id='Checking Account', type='Concept', properties={}), Node(id='Savings Account', type='Concept', properties={}), Node(id='Money Market Account', type='Concept', properties={}), Node(id='Money Market Fun

In [10]:
graph.add_graph_documents(data)